In [1]:
# Checked By Chaojie Wang 2018/10/8
# Used For IMDB Precessing

import codecs
import copy
import numpy as np

def clean_content(imdb_str):
    
    imdb_str = ' ' + imdb_str + ' '
    imdb_word_split = []
    count_flag      = 0
    imdb_str        = imdb_str.lower() # 大小写转换
    
    for i in range(len(imdb_str) - 1):

        if(imdb_str[i] is ' ')and(imdb_str[i+1] is not ' ')and(count_flag is 0):
            start_index = i+1
            count_flag = 1

        if(imdb_str[i] is not ' ')and(imdb_str[i+1] is ' ')and(count_flag is 1):
            end_index = i+1
            imdb_word_split.append(imdb_str[start_index:end_index])
            count_flag = 0
    return imdb_word_split

imdb_train_list = []
with open('/home/xiaosucheng/Desktop/ICML_2019/data_process/IMDB/imdb-train.txt.tok','r') as f:  #注意open 和 codecs.opem
    line = f.readlines()
    imdb_train_list.extend(line)
    
imdb_train_label = []
with open('/home/xiaosucheng/Desktop/ICML_2019/data_process/IMDB/imdb-train.cat','r') as f:
    line = f.readlines()
    imdb_train_label.extend(line)
    
imdb_test_list = []
with open('/home/xiaosucheng/Desktop/ICML_2019/data_process/IMDB/imdb-test.txt.tok','r') as f:  #注意open 和 codecs.opem
    line = f.readlines()
    imdb_test_list.extend(line)    
    
imdb_test_label = []
with open('/home/xiaosucheng/Desktop/ICML_2019/data_process/IMDB/imdb-test.cat','r') as f:
    line = f.readlines()
    imdb_test_label.extend(line)
    
# Label
for i in range(len(imdb_train_label)):
    imdb_train_label[i] = imdb_train_label[i].rstrip('\n')
    
for i in range(len(imdb_test_label)):
    imdb_test_label[i] = imdb_test_label[i].rstrip('\n')
    
imdb_class = []
for i in range(len(imdb_train_label)):
    if imdb_train_label[i] not in imdb_class:
        imdb_class.append(imdb_train_label[i]) 

imdb_train_split       = []
imdb_train_label_index = []
for i in range(len(imdb_train_list)):
    imdb_train_split.append(clean_content(imdb_train_list[i].rstrip('\n')))
    imdb_train_label_index.append(imdb_class.index(imdb_train_label[i]))
    
imdb_test_split       = []
imdb_test_label_index = []
for i in range(len(imdb_test_list)):
    imdb_test_split.append(clean_content(imdb_test_list[i].rstrip('\n')))
    imdb_test_label_index.append(imdb_class.index(imdb_test_label[i]))
    
# calculate vab
imdb_vab_count = {}

for i in range(len(imdb_train_split)):
    imdb_str = imdb_train_split[i]
    for word_index in range(len(imdb_str)):
        if (not imdb_vab_count.has_key(imdb_str[word_index])):
            imdb_vab_count.update({imdb_str[word_index]: 1})
        else:
            imdb_vab_count[imdb_str[word_index]] = imdb_vab_count[imdb_str[word_index]] + 1
            
del_stop_word = False
if del_stop_word is True:
    # stop word list
    stop_word_list = []
    with open('/home/xiaosucheng/Desktop/ICML_2019/data_process/IMDB/imdb_stop_word.txt','r') as f:  #注意open 和 codecs.opem
        line = f.readlines()
        for i in range(len(line)):
            stop_word_list.append(line[i].rstrip('\n'))

    for i in range(len(stop_word_list)):
        if imdb_vab_count.has_key(stop_word_list[i]):
            imdb_vab_count.pop(stop_word_list[i])
    print 'delete stop word'       
    
# sort vab
imdb_vab                  = {}
imdb_vab_list_orgin       = imdb_vab_count.keys()  # list
imdb_vab_count_list_orgin = imdb_vab_count.values() # list
imdb_vab_list             = []
imdb_vab_count_list       = []

imdb_vab_index      = np.argsort(-np.array(imdb_vab_count_list_orgin))  # np.argsort(-x) 降序排序
for i in range(len(imdb_vab_list_orgin)):
    imdb_vab.update({imdb_vab_list_orgin[imdb_vab_index[i]]: i + 1})    # 0 给stop_word　所以+1
    imdb_vab_list.append(imdb_vab_list_orgin[imdb_vab_index[i]])
    imdb_vab_count_list.append(imdb_vab_count_list_orgin[imdb_vab_index[i]])

print 'calculate vocabulary finished'

vab_size = 30000 #包含stop_word_index　共有 30000个
# word2index 如果不含有该词或者index超过vab_size,则置成０
imdb_train_list_index = copy.deepcopy(imdb_train_split)
for i in range(len(imdb_train_split)):
    imdb_str = imdb_train_split[i]
    for word_index in range(len(imdb_str)):
        if (not imdb_vab.has_key(imdb_str[word_index])) or (imdb_vab[imdb_str[word_index]] >= vab_size):
            imdb_train_list_index[i][word_index] = 0
        else:
            imdb_train_list_index[i][word_index] = imdb_vab[imdb_str[word_index]]
print 'train word2index finished'

imdb_test_list_index = copy.deepcopy(imdb_test_split)
for i in range(len(imdb_test_split)):
    imdb_str = imdb_test_split[i]
    for word_index in range(len(imdb_str)):
        if (not imdb_vab.has_key(imdb_str[word_index])) or (imdb_vab[imdb_str[word_index]] >= vab_size):
            imdb_test_list_index[i][word_index] = 0
        else:
            imdb_test_list_index[i][word_index] = imdb_vab[imdb_str[word_index]]
print 'test word2index finished'

# imdb_vab_list 　　　　　 词表的词
# imdb_vab_count_list 　  词表词的词频

import cPickle
IMDB = {}
IMDB['Vocabulary']       = imdb_vab_list
IMDB['Vab_count']        = imdb_vab_count_list
IMDB['Train_Origin']     = imdb_train_list
IMDB['Train_Label']      = imdb_train_label_index
IMDB['Train_Word_Split'] = imdb_train_split
IMDB['Train_Word2Index'] = imdb_train_list_index
IMDB['Test_Origin']      = imdb_test_list
IMDB['Test_Label']       = imdb_test_label_index
IMDB['Test_Word_Split']  = imdb_test_split
IMDB['Test_Word2Index']  = imdb_test_list_index
cPickle.dump(IMDB, open('./IMDB_30k.pkl','wb'))
print 'save finished'

calculate vocabulary finished
train word2index finished
test word2index finished
save finished
